# Highlights

*Securely trace patient location history to privately notify those who may have been near.*

1. A plan to provide contact tracing using apps and devices people already trust. 
  1. Users only need to opt-in, trusting partner app providers (e.g. Apple Maps, Google Maps, phone carriers, WeChat), to handle sensitive data.
  1. For added privacy, users can keep identifying information on their device only.
1. Details a distributed solution, using strong encryption.
  1. Single-use numbers, cryptographically random, would function as person ID for each location history entry. Users can specify which agencies/authorities to trust with the ability to decrypt subsets of their data. Newly diagnosed patients can choose to share encrypted person IDs stored temporarily on their devices, to identify overlap in recent locations (encrypted and stored in the cloud). 
  1. Users who were also at those locations at the same time could only be notified by *polling*; it should be possible for them and only them to know that they were at some undisclosed locations that overlapped with a patient.
1. Estimates monthly storage costs (USD) for 7B people:
  1. No outbreak, logging every 15 minutes: &#0036;150K (7 Petabytes)
  1. 10K active patients, logging every minute: &#0036;2.5M (120 Petabytes)
  1. 100K active patients, logging every minute: &#0036;3.3M (150 Petabytes)


# Update: 4/2/2020 - Code open-sourced

1. Python library shared on [ninja-pad-python repo](https://github.com/dantam/ninja-pad-python)
  1. Demonstrates feasibility simulating people movement.
  1. Creates public/private keys, sqlite databases, and config files for full cycle demo.

# Main differences from actual contact tracing ([Apple and Google](https://techcrunch.com/2020/04/10/apple-and-google-are-launching-a-joint-covid-19-tracing-tool/))
1. Ability to opt-in retroactively - location data can be shared after the fact, but anonymized contacts cannot.
1. Ability to handle environmental contamination - indirect infections can be estimated from location data.
1. Ability to use existing location data, without new apps or operating system updates.

# Background

To prevent a virus from spreading, one important component is to notify people who have a high chance of being recently infected, to self-quarantine and/or get tested. This is known as contact tracing. People who have come into contact with diagnosed persons are more likely to be also infected. One way to contact trace is to trace the locations a new patient has been in.

However, most people do not want their location history public. This notebook is a proposal to share location in a cryptographically strong way, while making it likely that a high percentage of people who should be notified would be. They would be notified by *polling*, their true identification being strongly encrypted.

See nytimes story for more background: https://www.nytimes.com/2020/03/19/us/coronavirus-location-tracking.html

# Privacy highlights

The proposal would make both location and personal identification mostly private:
1. Trusted location and personal "authorities" could separately decrypt some of the information. (More than one authority could be used by each person for each type of information).
1. Full personal identification is possible only with encrypted information on the device. 
1. The on-device store would destroy entries older than a set time, such that data from earlier than some number of days would be untraceable.
1. Only when trusted medical providers indicate a person has been infected, would it be possible for "authorities" to link that particular person to their location history.
1. Unless "location authority" is the same as "personal authority" for a particular log entry, the "authorities" would not know who appeared where.

If a user already trusts a provider (e.g. map app) with their location data, the provider could act as an "on-device store". That way a user could opt in to this service by simply opting in from their existing trusted app(s). Note, if the trusted provider acts as an on-device store, the user would be essentially *push-notified* by the trusted provider.

See [architectural diagram](files/ninja_pad.png) (also included below).

# Related Work
1. Private Kit
  1. From [MIT](http://privatekit.mit.edu/); open sourced: [github](https://github.com/tripleblindmarket/private-kit)
  1. Logs location once every five minutes, stores 28 days of data in under 100KB. Location data can only be accessed on device. Data transfer occurs only if the user chooses to share using a QR code.
  1. Main differences: 
    1. location privacy - accessible on device
    1. location privacy - requires redacted publication
1. Trace Together
  1. [Singapore Government official page](https://www.tracetogether.gov.sg/); [FAQ on data protection](https://tracetogether.zendesk.com/hc/en-sg/articles/360043234694-How-is-my-data-protected-)
  1. Records proximity via Bluetooth of nearby app users, logs contact using secret ID provided by Government.
  1. Main differences: 
    1. personal privacy - requires centralized ID mapping
    1. location efficiency - requires human tracers to obtain location information
    1. location privacy - requires broadcast to identify possible transmission through items in location
    1. bluetooth may work not work for [environmental contamination](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC539564/)
1. Many other groups are also working on this:
    1. [Pepp-PT](https://www.pepp-pt.org/)
    1. [CoEpi](https://github.com/Co-Epi/CEN/blob/main/protocol.md)
    1. [Covid Watch - Stanford](https://www.covid-watch.org/article#contactTracing)
    1. [OpenMined](https://blog.openmined.org/covid-app-privacy-advice/)
1. Two promising techniques that should be explored more are [private set intersection and homomorphic encryption](https://security.googleblog.com/2019/06/helping-organizations-do-more-without-collecting-more-data.html)


# Non-private illustration

A pseudocode scratch of a non-privacy-aware module would help illustrate how people tracking could work. Privacy issues and solutions would follow.

In [1]:
# pseudo code to show basic notification workflow, using location and time

from collections import defaultdict

class NonPrivateTracker:
    def __init__(self, time_interval_to_keep='2 weeks'):
        self.location_log = defaultdict(list)
        self.person_to_spacetimes = defaultdict(list)
        self.spacetimes_to_persons = defaultdict(list)
        self.time_interval_to_keep = time_interval_to_keep
        
    def log(person, location, time):
        self.location_log[time].append((person, location))
        
    def process_log(time_window):
        # truncate entries older than time_interval_to_keep
        for time in time_window:
            for person, location in self.location_log[time]: 
                self.person_to_spacetimes[person].append((time, location))
                self.spacetimes_to_persons[(time, location)].append(person)

    def notify(new_carrier, time_window):
        spacetimes = self.person_to_spacetimes[new_carrier]
        to_notify = set()
        for log_time, location in spacetimes:
            if log_time in time_window:
                persons = self.spacetimes_to_persons[(log_time, location)]
                to_notify.add(persons)
        return to_notify - new_carrier

Essentially this tracker works as follows:
1. Maintain a log of append-only location entries, where each entry contain tuple `(time, location, person)`
1. Periodically the log would be processed, to make further queries fast, namely:
  1. O(num_location_entries_for_new_carrier) * O(num_colocated_people_with_new_carrier_in_specific_time_window)
1. If the log is periodically truncated, so that entries older than some time frame are removed, the run time estimate above would only depend on the second term, number of colocated people. If location granularity is relatively high, it seems reasonable that fewer than 100 people could share the same location at any time. That means the run time operation is constant time, O(1). Offline operation, log processing, can be done in parallel using map reduce.
1. Reported time would also have to be in some agreed-upon values, perhaps in 15 minute intervals starting from 00:00.

# Privacy issues includes:

1. Person identification is exposed - anyone can see where a person was at any time, for any person, given the log. This is not ideal for the person, as they may have gone to places they do not want to be public. (For example, a person may want their business address to not be publicly associated with their personal identity).
2. Location identification is exposed - anyone can see how many people each location had at any time, given the log. This is not ideal for the location owner, as they may not want traffic patterns to those locations to be known. (For example, a restaurant may want to keep the number of customers private).

# Requirements

This proposals require three publicly available log files (in the cloud):
1. Location Log
1. Contamination Log
1. Notification Log

Additional requirements:
1. Identification and public keys authorities (Personal, Location, Medical).
1. Privacy enforcers (to batch, scramble, and hide network traces).


# Diagram

![workflow](files/ninja_pad.png)

## Message format:

In the description below, public, encrypted messages, including notification from *polling*, are formatted as:

``` diff 
- red ``` 

On-device storage messages are formatted as:
``` diff
+ green ``` 

## "One time pad" here is not what cryptographers usually mean
This notebook refers to ["one time pads"](https://en.wikipedia.org/wiki/One-time_pad). In cryptography, "one time pad" is "an encryption technique that cannot be cracked", by using pre-shared keys. In this notebook, "one time pad" only means "single use cryptographically random numbers". The "one time pad" is *not* used as a key for further encryption, but rather as a tracing indicator that is shared by various components. 

# Add Personal privacy

To solve for personal privacy, I propose the following:
1. Instead of logging identifiable person tokens in the tuple `(time, location, person)`, use a cryptically random number (one time pad), for example: by `secrets.token_hex(256)`.
1. One time pads should be stored on user device only, along with timestamp used
1. Periodically pads used earlier than some time interval must be purged
1. The log entry (before location encryption) would be: `(time, location, encrypted_one_time_pad)`;<br/> On-device store would hold:

```diff
+ (time, \
+  one_time_pad, \
+  PersonalAuthority)
```

# Add Location privacy

To solve for location privacy, while allowing broadcasts of contaminated `(time, location)`s. I propose the following:
1. Instead of logging location tokens in the tuple `(time, location, person)`, encrypt location with the public keys of one or more LocationAuthority's
1. People already trust some organizations for their location history, for example, when using a GPS-based map application. 
1. The tracker app can only detect collocation when location is shared to at least one LocationAuthority's. As such, it should be enabled to share location 24 hours a day by default.
1. The tracker app may want to enable a user to block times and/or locations where they do not want to be tracked at all, for example, every night from 11pm to 6am. The user would then be responsible to notify people they often see in the untracked time-spaces.
1. [Cryptographically strong padding](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/#cryptography.hazmat.primitives.asymmetric.padding.OAEP) is used, such that the same location encrypted twice would appear different
1. The log entry would be:

```diff
- (time, \
-  encrypted_location, \
-  encrypted_one_time_pad)
```

Some further precautions:
1. log entries from device should be submitted in batch, possibly from VPNs to hide originating location from network traces
1. log entries are to be published by PrivacyEnforcers's after aggregating and scrambling a large number of users and entries.

# Notify collocators by polling

When a user is diagnosed, they could share their on-device store entries with the PersonalAuthority's. (This could be further restricted by trusted MedicalAuthority's only). 

The PersonalAuthority's would:
1. Find the `encrypted_one_time_pad` in the common logs for this user, by decrypting the encrypted_one_time_pad's with their private keys. Decrypted items must match on-device one_time_pad's. If they do not match, there are no guarantees those log entries originated from the user.
1. Update the log that the encrypted_location was contaminated at the associated time, by appending a log entry:

```diff
- (time, \
-  encrypted_location, \
-  PersonalAuthorityID)
```

LocationAuthority's would scan the log for needed action:
1. Local Authority would attempt to decrypt the encrypted_location. If decryption succeeds, the location was previously encrypted using the LocationAuthority's public key. As such, this particular LocationAuthority was responsible to decrypt the encrypted_location.
1. It should broadcast privately to other LocationAuthority's that there was a contamination event, at `(time, other_encrypted_location)`, by appending multiple log entries like: 

```diff
- (time, \
-  other_encrypted_location, \
-  LocationAuthorityID)
```
The contaminated location is encrypted multiple times, using each known LocationAuthority's public keys. 

LocationAuthority's would:
1. Treat the above broadcasted messages the same way as though a PersonalAuthority told them `(time, encrypted_location) `was contaminated, except no further broadcast was needed.
1. Decrypt location as needed, then along with passed in time field, look for other log entries encrypted using its public key. 
1. Any colocated person would have a log entry keyed by `(time, encrypted_location)`. The `encryted_one_time_pad` used for that entry would then allow a user to know they could be contaminated.

To notify:
1. This would be done in a <b>separate log file</b>, keeping known entries like so:

```diff
- (time, encrypted_one_time_pad)
```
1. A user's location tracker app could check it's on-device storage for any matches, which would indicate its owner may be infected. The owner should then self-quarantine and/or get tested.

# Considerations for nosy external agents

1. Police-state?
  1. If an external agent (e.g. police) wants to track where a person went, the published log itself won't help, because both personal and locational information are encrypted. 
  1. Without access to an on-device store, but with access to the private keys of all the Location and Personal Authorities, an external agent could detect whether a location had at least one person at a time, and can estimate a plausible range of number of people there. However, the agent could not figure out who exactly, because the only available personal identification would be a single-use random number. 
  1. With access to the on-device store, and subpoenas at the Location Authorities used, it is possible to track where a person went, provided the app was enabled in the time window of interest, and the time window was within the `time_interval_to_keep` as enforced in the on-device store. 
  1. Since a user can choose which "authorities" to trust, they could select ones hosted in other jurisdictions. For example, a spy operating in a country could pick from a set of "authorities" in other countries. This is especially true for "privacy enforcer".

1. Home church? Area 51?
  1. If periodic meetings at the same location need to stay secret, users could stop the tracking app from sharing their location around the time or space. Otherwise, the log lines could leak an increase of activity in an area. Note, this does not help against other tracking devices like cameras.
  1. A more secure way is to host special LocationAuthority's. Users would choose to always use these LocationAuthority's when they are within a much larger area (e.g. district vs building). A sudden drop in activity surrounding a small area may leak information about desire for secrecy.

# Cost estimates

| Monthly Estimates	for the Entire World's population (7B) | Reporting every 15 minutes, assuming no outbreak | Reporting every minute, with 10K active patient cases | Reporting every minute, with 300K active patient cases |
| --- |---|---| --- |
| Storage (TB) | 7,338 | 110,259 | 121,657 |
| Network (TB) | 0 | 729 | 30,482 |
| Storage Cost (USD) | &#0036;154,103 | &#0036;2,315,429 | &#0036;2,554,791 |
| Network Cost (USD) | &#0036;0 | &#0036;29,147 | &#0036;762,048 |
| Total Cost (USD) | &#0036;154,103 | &#0036;2,344,576 | &#0036;3,316,839 |
| On device storage, per person (MB) | 0.13 | 1.94 | 1.94 |


More granular estimates on [Google Sheet](https://docs.google.com/spreadsheets/d/1ruBybgc34bmuqcrFFYUwhPo6k5FAuIPaVpQlFFcUDw0/edit?usp=sharing)


# Further considerations

1. Adversarial logging
  1. An adversarial agent may log bogus entries, to (a) cause panic in a target region, (b) overwhelm the system to prevent normal functioning.
  1. Aggregate-level signing and checking could be added, to adjust the amount of trust and urgency for each log entry. For example, an entry in the Contamination Log could be signed by (a) user app build, (b) medical provider app build, (3) medical authority organization and/or aggregated geographical location, (4) person authority. If log processing is overwhelmed, further processing could be delayed if one or more parts of the signing chain appear abusive.
  1. Panic is unlikely because final user recommendation can be controlled, based on actual outbreak observations and trust in the signing parties. A recommendation of "get tested" should always be based on strong evidence, such that bogus log entries can only do so much harm.
1. Logs organization
  1. Centralized cloud providers such as AWS may be well suited for this.
  1. Entries could be bucketed by time and hash of some agreed upon parts of the encrypted payload.
1. Probabilistic reflections
  1. Much in the system carries uncertainties: aside from the trustworthiness of logging agents mentioned above, the accuracy of medical tests, the specificities of location and time, are both obvious factors. Final recommendations (e.g. no-notification, self-quarantine, get-tested) should be adjusted based on the level of confidence.

# Adoption

1. Try to gain support from big players (Apple, Facebook, Google, etc) who already have location sensitive data. Most users are probably OK with these big players acting as their "device" for this service.
1. Publish protocol, open source code, and certify as [openly operated services](https://openlyoperated.org/how-to) to show viable alternatives to big players.
1. Ask people to opt-in using existing apps, or install new standalone app.